### Import the libraries

In [34]:
# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

# Experiment tracking
import mlflow

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Set up experiment tracking with MLFlow

In [2]:
# Store experiment files on the database
# mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Create a new experiment
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('file:///c:/Users/ffirman/Documents/SQL Server Management '
 'Studio/Research/Data Science/mlops-zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1718350738258, experiment_id='1', last_update_time=1718350738258, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

### Helper functions

In [35]:
# Transform nyc-taxi dataset
def read_dataframe(file_path):

    # Load csv dataset
    df = pd.read_parquet(file_path)

    # Convert pick up and drop off location into string format
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    # Convert pick up and drop off time column into datetime format
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    # Add duration column, differences between pick up and drop off time
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    # Convert duration column into minute format
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter duration between 1 and 60 minutes
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [36]:
# Load data train and validation
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

print(f'Dimension of data train : {df_train.shape[0]}, {df_train.shape[1]}')
print(f'Dimension of data validation : {df_val.shape[0]}, {df_val.shape[1]}')

Dimension of data train : 73908, 21
Dimension of data validation : 61921, 21


### Data preparation

Feature engineering experimentation

In [37]:
# Feature engineering to concatenate pick up and drop off location id
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

Preparation for data train and data validation

In [38]:
# Define categorical and numerical columns
categorical = ['PU_DO']
numerical = ['trip_distance']

# Instantiate dictionary vectorizer
dv = DictVectorizer()

# Fit and transform data train into vectors with feature-value format
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Transform data validation into vectors based on pattern got from data train
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Define target column for predicting
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

### Modelling with MLflow

#### 1. Experiment without MLflow

Linear regression

In [39]:
# Linear regression
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715205808363

Lasso regression

In [40]:
# Lasso regression
lasso = Lasso()
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

12.212583224318818

Ridge regression

In [41]:
# Ridge regression
ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.703735145845265

#### 2. Experiment tracking with single parameter

Linear Regression

In [15]:
# Linear regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "linear-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "data/green_tripdata_2021-02.parquet")
    
    # Training model
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    # mlflow.sklearn.log_model(lr, 'Linear Regression')
    
    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

Lasso Regression

In [16]:
# Lasso regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "lasso-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    # Training model
    l1 = Lasso(alpha=alpha)
    l1.fit(X_train, y_train)
    # mlflow.sklearn.log_model(l1, 'Lasso Regression')
    
    y_pred = l1.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

Ridge Regression

In [17]:
# Ridge regression

with mlflow.start_run():
    
    # Log general info, ex : developer_name
    mlflow.set_tag("developer", "farel")
    mlflow.set_tag("model", "ridge-regression")
    
    # Log parameters
    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    # Training model
    l2 = Ridge(alpha=alpha)
    l2.fit(X_train, y_train)
    # mlflow.sklearn.log_model(l2, 'Ridge Regression')
    
    y_pred = l2.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)

    # Keep info related to metrics resulted from the model
    mlflow.log_metric("rmse", rmse)

#### 3. Experiment tracking with multiple parameters 

Ridge Regression

In [18]:
# Ridge regression
alphas = np.arange(0, 1, 0.1)
for alpha in alphas:
    with mlflow.start_run():
        # Log general info, ex : developer_name
        mlflow.set_tag("developer", "farel")
        mlflow.set_tag("model", "ridge-regression")
        mlflow.set_tag("experiment", "multiple-parameters")        

        # Log a parameter
        mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
        mlflow.log_param("validation-data-path", "data/green_tripdata_2021-02.parquet")
        mlflow.log_param("alpha", alpha)
        
        # Train model
        l2 = Ridge(alpha=alpha)
        l2.fit(X_train, y_train)
        # mlflow.sklearn.log_model(l2, 'Ridge Regression')
        
        # Evaluate model
        y_pred = l2.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Log metrics
        mlflow.log_metric("rmse", rmse)

#### 4. XGBoost Model

Data Preparation

In [42]:
# Convert data train and validation into optimized DMatrix format
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [30]:
# Define objective function for training xgboost model using MLflow
def objective(params):
    with mlflow.start_run():
        
        # Log general info
        mlflow.set_tag("developer", "farel")
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag("experiment", "tuning-hyperparams")        

        # Log batch of parameters
        mlflow.log_params(params)

        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        # Evaluate model
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)

        # Log metrics
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [31]:
# Define hyperparameters combination
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

Training Model

In [32]:
# Call objective function to running xgboost model
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials(),
    verbose=False
)

[0]	validation-rmse:11.19695                          
[1]	validation-rmse:10.34479                          
[2]	validation-rmse:9.63415                           
[3]	validation-rmse:9.04502                           
[4]	validation-rmse:8.56366                           
[5]	validation-rmse:8.16366                           
[6]	validation-rmse:7.84257                           
[7]	validation-rmse:7.58258                           
[8]	validation-rmse:7.36830                           
[9]	validation-rmse:7.19689                           
[10]	validation-rmse:7.05864                          
[11]	validation-rmse:6.94798                          
[12]	validation-rmse:6.85769                          
[13]	validation-rmse:6.78583                          
[14]	validation-rmse:6.72414                          
[15]	validation-rmse:6.67520                          
[16]	validation-rmse:6.63500                          
[17]	validation-rmse:6.60255                          
[18]	valid

Find Best Parameters

In [32]:
# Find best parameters from run experiment
best_run_id = mlflow.search_runs(
    filter_string="tags.model='xgboost' and tags.experiment='tuning-hyperparams'",
    order_by=["metrics.rmse"],
    max_results=1
).run_id

best_params = mlflow.get_run(best_run_id[0]).data.params
best_params


{'learning_rate': '0.34773952198026464',
 'max_depth': '91',
 'min_child_weight': '1.9487002778399356',
 'objective': 'reg:linear',
 'reg_alpha': '0.04463705727219639',
 'reg_lambda': '0.02039743730465589',
 'seed': '42'}

Training Model with Best Parameters

In [44]:
# Do autologging using mlflow built-in function
mlflow.xgboost.autolog()

# Train model
booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)    

2024/06/14 17:34:57 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '7b608f7cebd641b2bdd53e0fd850f47f', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:9.50445
[1]	validation-rmse:8.02529
[2]	validation-rmse:7.26001
[3]	validation-rmse:6.87238
[4]	validation-rmse:6.67995
[5]	validation-rmse:6.57614
[6]	validation-rmse:6.51571
[7]	validation-rmse:6.47955
[8]	validation-rmse:6.45735
[9]	validation-rmse:6.44171
[10]	validation-rmse:6.43150
[11]	validation-rmse:6.42387
[12]	validation-rmse:6.41853
[13]	validation-rmse:6.41447
[14]	validation-rmse:6.41176
[15]	validation-rmse:6.40921
[16]	validation-rmse:6.40627
[17]	validation-rmse:6.40304
[18]	validation-rmse:6.39914
[19]	validation-rmse:6.39495
[20]	validation-rmse:6.39196
[21]	validation-rmse:6.38966
[22]	validation-rmse:6.38732
[23]	validation-rmse:6.38453
[24]	validation-rmse:6.38277
[25]	validation-rmse:6.38055
[26]	validation-rmse:6.37790
[27]	validation-rmse:6.37675
[28]	validation-rmse:6.37550
[29]	validation-rmse:6.37442
[30]	validation-rmse:6.37114
[31]	validation-rmse:6.36999
[32]	validation-rmse:6.36848
[33]	validation-rmse:6.36734
[34]	validation-rmse:6.3